In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [25]:
class RidgeRegGradientDescentMulti:
    def __init__(self):
        self.cost=0
    def fit(self,X,Y,Debug=0,trials=10,alpha=0.1,early_stop=0,l2=0):
        self.__init__()
        
        self.best_params=np.zeros([len(X[0])+1,])
        self.direcs=[True]*(len(X[0])+1)
        
        if Debug==1:
            print("Debug:X.shape:",X.shape)
            print("Debug:Y.shape:",Y.shape)
        for t in range(trials):
            
            if t==0:
                self.cost=0
                for i in range(len(X)):
                    residuals=0
                    residuals+=Y[i]
                    for j in range(len(X[0])+1):
                        if j==0:
                            residuals-= self.best_params[j]
                        else:
                            residuals-= self.best_params[j]*X[i][j-1]
                    
                    self.cost+=residuals**2
                self.cost+=sum(self.best_params[1:3]**2)
                if Debug==1:print("Debug:sum of cost is",self.cost)
            #initial direction checking
            if t==0:
                positiveDirecCost=np.zeros([len(X[0])+1,])
                negativeDirecCost=np.zeros([len(X[0])+1,])
                for i in range(len(X[0])+1):
                    for j in range(len(X)):
                        pos_residuals=0
                        neg_residuals=0
                        pos_residuals+=Y[j]
                        neg_residuals+=Y[j]
                        for k in range(len(X[0])+1):
                            if k==i:
                                if k==0:
                                    pos_residuals-= (self.best_params[k]+alpha)
                                    neg_residuals-= (self.best_params[k]-alpha)
                                else:
                                    pos_residuals-= (self.best_params[k]+alpha)*X[j][k-1]
                                    neg_residuals-= (self.best_params[k]-alpha)*X[j][k-1]
                            else:
                                if k==0:
                                    pos_residuals-= self.best_params[k]
                                    neg_residuals-= self.best_params[k]
                                else:
                                    pos_residuals-= self.best_params[k]*X[j][k-1]
                                    neg_residuals-= self.best_params[k]*X[j][k-1]     
                        positiveDirecCost[i]+=pos_residuals**2
                        negativeDirecCost[i]+=neg_residuals**2
                    for j in range(len(X[0])+1):
                        if j!=0 and j==i:
                            positiveDirecCost[i]+=l2*((self.best_params[j]+alpha)**2)
                            negativeDirecCost[i]+=l2*((self.best_params[j]-alpha)**2)
                        else:
                            if j!=0:
                                positiveDirecCost[i]+=l2*(self.best_params[j]**2)
                                negativeDirecCost[i]+=l2*(self.best_params[j]**2)
                for i in range(len(X[0])+1):
                    if positiveDirecCost[i] <=negativeDirecCost[i]:
                        self.direcs[i]=True
                    if negativeDirecCost[i] < positiveDirecCost[i]:
                        self.direcs[i] =False
                if Debug==1:print(self.direcs)
                
            #parameter updates
            for i in range(len(X[0])+1):
                convergence=0
                proposal_cost=0
                for j in range(len(X)):
                    prop_residuals=0
                    prop_residuals+=Y[j]
                    for k in range(len(X[0])+1):
                        if k==i:
                            if k==0:
                                if self.direcs[i]==True: prop_residuals-= (self.best_params[k]+alpha)
                                if self.direcs[i]==False: prop_residuals-= (self.best_params[k]-alpha)                                        
                            else:
                                if self.direcs[i]==True: prop_residuals-= (self.best_params[k]+alpha)*X[j][k-1]
                                if self.direcs[i]==False: prop_residuals-= (self.best_params[k]-alpha)*X[j][k-1]  
                        else:
                            if k==0:
                                prop_residuals-= self.best_params[k]
                            else:
                                prop_residuals-= self.best_params[k]*X[j][k-1]   
                    proposal_cost+=prop_residuals**2
                    for j in range(len(X[0])+1):
                        if j!=0 and j==i:
                            if self.direcs[i]==True: proposal_cost+=l2*((self.best_params[j]+alpha)**2)
                            if self.direcs[i]==False: proposal_cost+=l2*((self.best_params[j]-alpha)**2)
                        else:
                            if j!=0:
                                if self.direcs[i]==True: proposal_cost+=l2*(self.best_params[j]**2)
                                if self.direcs[i]==False: proposal_cost+=l2*(self.best_params[j]**2)
                
                
                if Debug==1:print("Debug:Proposal Cost is ",proposal_cost)
                if self.cost > proposal_cost:
                    if self.direcs[i]==True: self.best_params[i]=self.best_params[i]+alpha
                    if self.direcs[i]==False: self.best_params[i]=self.best_params[i]-alpha
                    self.cost=proposal_cost
                    convergence+=1
            if convergence==0 and early_stop==1:
                print("early stop")
                break
            else:
                continue
            break
        print("Best parameters:",self.best_params)
        print("Min Cost:",self.cost)
        return self.best_params

In [8]:
def gendata_linear(n,mean,cov,w,intercept,sigma):
    x = np.random.multivariate_normal(mean,cov,n)
    y = np.zeros(n)
    for i in range(n):
        y[i] = w@x[i,:] + intercept + np.random.randn()*sigma
    return x,y

In [30]:
numsample = 100
mean = [0,0,0]
cov = [[1,0.99,0],[0.999,1,0],[0,0,1]]
w = [0.01,1,2]
sigma = 1
intercept = 0
x,y = gendata_linear(numsample,w,cov,w,intercept,sigma)

C:\Users\shota\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: covariance is not positive-semidefinite.
  


In [54]:
reg=RidgeRegGradientDescentMulti()
reg.fit(x,y,Debug=0,alpha=0.1,trials=100,early_stop=0,l2=100)

Best parameters: [4.6 0.  0.1 0.1]
Min Cost: 772.3940481999364


array([4.6, 0. , 0.1, 0.1])